<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/BubbleDewPT_inclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.py

In [ ]:
import numpy as np
from scipy.optimize import root
import che as che
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'

In [ ]:
p = che.Props(['Benzene','Toluene'])

In [ ]:
def bubbleP(x,T):
    return np.sum(x*p.Pvap(T))

In [ ]:
def bubbleT(x,P):

    def eq(v):
        T=v[0]
        return [bubbleP(x,T) - P]

    return root(eq, [298.]).x[0]

In [ ]:
def dewP(y,T):
    return 1./np.sum(y/p.Pvap(T))

In [ ]:
def dewT(y, P):

    def eq(v):
        T=v[0]
        return [dewP(y,T)-P]
    
    return root(eq, [298.]).x[0]

In [ ]:
z1 = np.linspace(0.,1.,11)
z2= 1-z1

T=273.15+120.

Psat = p.Pvap(T)
bubbleP_list = z1*Psat[0] + z2*Psat[1]

In [ ]:
dewP_list=[]
for y1 in z1:
    y2=(1-y1)
    y=np.array([y1,y2])
    dewP_list.append(dewP(y,T))

In [ ]:
P=150e3
bubbleT_list=[]
dewT_list=[]
for xy1 in z1:
    xy2=(1-xy1)
    bubbleT_list.append(bubbleT(np.array([xy1,xy2]), P))
    dewT_list.append(dewT(np.array([xy1,xy2]),P))

In [ ]:
fig=make_subplots(rows=2,cols=2, subplot_titles=[f'T={T:.0f} K', f'P={P:.0f} Pa', f'T={T:.0f} K', f'P={P:.0f} Pa'])
fig.add_scatter(x=z1, y=bubbleP_list,row=1,col=1, name='bubbleP')
fig.add_scatter(x=z1, y=dewP_list,row=1,col=1, name='dewP')
fig.add_scatter(x=z1, y=bubbleT_list,row=1,col=2, name='bubbleT')
fig.add_scatter(x=z1, y=dewT_list,row=1,col=2, name='dewT')
fig.add_scatter(x=z1, y=z1*p.Pvap(T)[0]/bubbleP_list, row=2, col=1, name='xy_Tconst' )

fig.add_scatter(x=z1, y=z1*np.array([p.Pvap(T)[0] for T in bubbleT_list])/P,
                row=2, col=2, name='xy_Pconst' )

fig.update_xaxes(tickvals=np.linspace(0,1,11), title='x1,y1')
fig.update_yaxes(title='Pressure (Pa)', row=1,col=1)
fig.update_yaxes(title='Temperature (K)', row=1,col=2)
fig.update_xaxes(title='x1', row=2,col=1)
fig.update_yaxes(title='y1', row=2,col=1)
fig.update_xaxes(title='x1', row=2,col=2)
fig.update_yaxes(title='y1', row=2,col=2)
fig.update_layout(title='Binary VLE for (1) Benzene and (2) Toluene', width=700, height=700)


In [ ]:
bubbleP_list

DeviceArray([131126.51502042, 147969.86177112, 164813.20852182,
             181656.55527251, 198499.90202321, 215343.24877391,
             232186.5955246 , 249029.9422753 , 265873.289026  ,
             282716.63577669, 299559.98252739], dtype=float64)

In [ ]:
#Rachford-Rice Calculation
T=393.
P=150e3
z = np.array([0.4, 0.6])
K = p.Pvap(T)/P
F = 10.  #moles of feed

def RR(v):
    VF=v[0]
    return [np.sum(z*(K-1)/(1+VF*(K-1)))]

In [ ]:
VF=root(RR,0.5).x[0]

In [ ]:
V=VF*F
L = F-V
x = z/(1+VF*(K-1))
y = K*x

In [ ]:
print(f'Total Vapor/Feed Ratio={VF}')
print(f'Total Vapor (moles)={V}')
print(f'Total Liquid (moles)={L}')
print(f'Liquid mole fractions: {x}')
print(f'vapor mole fractions: {y}')

Total Vapor/Feed Ratio=2.4863596200296993
Total Vapor (moles)=24.863596200296993
Total Liquid (moles)=-14.863596200296993
Liquid mole fractions: [0.1155665 0.8844335]
vapor mole fractions: [0.22996407 0.77003593]
